# BERT Pre-training and Fine-tuning

## Preparation

First, let's import necessary modules.

Note that utils.py includes some Blocks defined in the previous transformer notebook

In [1]:
import random, math

import d2l
import numpy as np
import mxnet as mx
from mxnet import gluon, nd
import gluonnlp as nlp

from utils import PositionalEncoding, MultiHeadAttention 
from utils import AddNorm, PositionWiseFFN, EncoderBlock
from utils import train_loop, predict_sentiment

### Encoder
<img src="../img/transformer-bert.png" alt="architecture" width="450"/>

### Segment Embedding

Different from the transformer encoder, the BERT encoder has an additional embedding for segment information.

<img src="../img/bert-embed.png" alt="seg embedding" width="750"/>

Similar to the Transformer encoder defined in the previous section, the BERT encoder has embeddings for words and positions. The `EncoderBlock` contains position-wise feed-forward network and self-attention blocks to encode inputs. For BERT, the newly added segment embedding captures the segment information of the input sentence pairs, used for the next sentence prediction task.


### BERT Encoder 

In [2]:
class BERTEncoder(gluon.nn.Block):
    def __init__(self, vocab_size, units, hidden_size,
                 num_heads, num_layers, dropout, **kwargs):
        super(BERTEncoder, self).__init__(**kwargs)
        # segment_embed for segment information
        self.segment_embed = gluon.nn.Embedding(2, units)
        self.word_embed = gluon.nn.Embedding(vocab_size, units)
        self.pos_encoding = PositionalEncoding(units, dropout)
        self.blks = gluon.nn.Sequential()
        for i in range(num_layers):
            self.blks.add(EncoderBlock(units, hidden_size, num_heads, dropout))

    def forward(self, words, segments, mask, *args):
        X = self.word_embed(words) + self.segment_embed(segments)
        X = self.pos_encoding(X)
        for blk in self.blks:
            X = blk(X, mask)
        return X

### Using the BERT Encoder

Now let's test the BERTEncoder with a data batch of 2 sentence pairs, each with 8 words. Random integers are used to represent words for demonstration purpose. For segment information, we use 0 to indicate the word comes from the first sentence, 1 to indicate the second setence.

In [3]:
encoder = BERTEncoder(vocab_size=30000, units=768, hidden_size=3072,
                      num_heads=12, num_layers=12, dropout=0.1)
encoder.initialize()

num_samples, num_words = 2, 8
# random words for testing
words = nd.random.randint(low=0, high=30000, shape=(num_samples, num_words))
# the corresponding segment information for each word
segments = nd.array([[0,0,0,0,1,1,1,1],[0,0,0,1,1,1,1,1]])

encodings = encoder(words, segments, None)
print(encodings.shape) # (batch_size, num_words, units)

(2, 8, 768)


### Next Sentence Classifier

Let us take a look at the first pre-training task: next sentence prediction. For this task, the encoding of the first token (the "[CLS]" token) is passed to a feed-forward network to make prediction.

Since next sentence prediction is a binary classification problem, we can use `SigmoidBinaryCrossEntropyLoss` as the loss function. In the following code block, we pass the encoding results to the `NSClassifier` to get the next sentence prediction. We use 1 as the label for true next sentence, and 0 otherwise. The prediction result and the label are then passed to the loss function for loss evaluation.

In [4]:
class NSClassifier(gluon.nn.Block):
    def __init__(self, units=768, **kwargs):
        super(NSClassifier, self).__init__(**kwargs)
        self.classifier = gluon.nn.Sequential()
        self.classifier.add(gluon.nn.Dense(units=units, flatten=False, activation='tanh'))
        self.classifier.add(gluon.nn.Dense(units=1, flatten=False))

    def forward(self, X, *args):
        X = X[:, 0, :]  # get the encoding of the first token
        return self.classifier(X)

ns_classifier = NSClassifier()
ns_classifier.initialize()

ns_pred = ns_classifier(encodings) # (batch_size, 1)
ns_label = nd.array([0, 1]) # 1 for true next setence, 0 otherwise
ns_loss_fn = gluon.loss.SigmoidBinaryCrossEntropyLoss()
ns_loss = ns_loss_fn(ns_pred, ns_label).mean()
print(ns_pred.shape, ns_loss.shape)

(2, 1) (1,)


### Masked Language Model Decoder

Masked language modeling is one of the two pre-training task, where random positions are masked and the model needs to reconstruct the masked words. In the masked language model decoder, we first use `gather_nd` to pick the dense vectors representing words at masked position. Then a feed-forward network is applied on them, followed by a fully-connected layer to predict the unnormalized score for all words in the vocabulary.

In [5]:
class MLMDecoder(gluon.nn.Block):
    def __init__(self, vocab_size, units, **kwargs):
        super(MLMDecoder, self).__init__(**kwargs)
        self.decoder = gluon.nn.Sequential()
        self.decoder.add(gluon.nn.Dense(units, flatten=False))
        # Gaussian Error Linear Units as the activation function [4]
        self.decoder.add(gluon.nn.GELU())
        self.decoder.add(gluon.nn.LayerNorm())
        # classification layer for `vocab_size` classes
        self.decoder.add(gluon.nn.Dense(vocab_size, flatten=False))

    def forward(self, X, masked_positions, *args):
         # gather encodings at mask positions
        X = nd.gather_nd(X, masked_positions)
        pred = self.decoder(X)
        return pred

### Using the Masked Language Model Decoder

In the following code block, we pass the encoding results to the `MLMDecoder` to get the masked language model prediction. We generate some random word indices as the label for demonstration purpose. For multi-class classification, we can use `SoftmaxCrossEntropyLoss` as the loss function. The prediction result and the label are then passed to the loss function for loss evaluation.

In [6]:
decoder = MLMDecoder(vocab_size=30000, units=768)
decoder.initialize()

mlm_positions = nd.array([[0,1],[4,8]])
mlm_label = nd.array([100, 200])
mlm_pred = decoder(encodings, mlm_positions) # (batch_size, vocab_size)
mlm_loss_fn = gluon.loss.SoftmaxCrossEntropyLoss()
mlm_loss = mlm_loss_fn(mlm_pred, mlm_label).mean()
print(mlm_pred.shape, mlm_loss.shape)

(2, 30000) (1,)


## BERT Fine-tuning (Sentiment Analysis)

In this section, we fine-tune the BERT Base model for sentiment analysis on the IMDB dataset.

### BERT for Sentence Classification

Let's first take
a look at the BERT model
architecture for single sentence classification below:

<img src="../img/bert-sa.png" alt="bert sentiment analysis" width="750"/>

Here the model takes a sentences and pools the representation of the first token in the sequence.
Note that the original BERT model was trained for a masked language model and next-sentence prediction tasks, which includes layers for language model decoding and
classification. These layers will not be used for fine-tuning sentence classification.

### Get Pre-train BERT Model

We can load the pre-trained BERT fairly easily using the model API in GluonNLP, which returns the vocabulary along with the model. We include the pooler layer of the pre-trained model by setting `use_pooler` to `True`.
The list of pre-trained BERT models available in GluonNLP can be found [here](../../model_zoo/bert/index.rst).

In [7]:
ctx = d2l.try_all_gpus()
bert_base, vocabulary = nlp.model.get_model('bert_12_768_12',
                                            dataset_name='book_corpus_wiki_en_uncased',
                                            pretrained=True, ctx=ctx,
                                            use_decoder=False, use_classifier=False)
print(bert_base)

BERTModel(
  (encoder): BERTEncoder(
    (dropout_layer): Dropout(p = 0.1, axes=())
    (layer_norm): BERTLayerNorm(eps=1e-12, axis=-1, center=True, scale=True, in_channels=768)
    (transformer_cells): HybridSequential(
      (0): BERTEncoderCell(
        (dropout_layer): Dropout(p = 0.1, axes=())
        (attention_cell): MultiHeadAttentionCell(
          (_base_cell): DotProductAttentionCell(
            (_dropout_layer): Dropout(p = 0.1, axes=())
          )
          (proj_query): Dense(768 -> 768, linear)
          (proj_key): Dense(768 -> 768, linear)
          (proj_value): Dense(768 -> 768, linear)
        )
        (proj): Dense(768 -> 768, linear)
        (ffn): BERTPositionwiseFFN(
          (ffn_1): Dense(768 -> 3072, linear)
          (activation): GELU()
          (ffn_2): Dense(3072 -> 768, linear)
          (dropout_layer): Dropout(p = 0.1, axes=())
          (layer_norm): BERTLayerNorm(eps=1e-12, axis=-1, center=True, scale=True, in_channels=768)
        )
        (la

### Model and Loss for Fine-tuning

Now that we have loaded the BERT model, we only need to attach an additional layer for classification.
The `BERTClassifier` class uses a BERT base model to encode sentence representation, followed by a `nn.Dense` layer for classification. We only need to initialize the classification layer. The encoding layers are already initialized with pre-trained weights.

In [8]:
class BERTClassifier(gluon.nn.Block):
    def __init__(self, bert, num_classes):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        # extra layer used for classification
        self.classifier = gluon.nn.Dense(num_classes)

    def forward(self, inputs, segment_types, seq_len):
        seq_encoding, cls_encoding = self.bert(inputs, segment_types, seq_len)
        return self.classifier(cls_encoding)

loss_fn = gluon.loss.SoftmaxCELoss()
net = BERTClassifier(bert_base, 2)
net.classifier.initialize(ctx=ctx)

## Data Preprocessing

To use the pre-trained BERT model, we need to:
- tokenize the inputs into word pieces,
- insert [CLS] at the beginning of a sentence, 
- insert [SEP] at the end of a sentence, and
- generate segment ids

### BERT-specific Transformations

We again use the IMDB dataset, but for this time, downloading using the GluonNLP data API. We then use the transform API to transform the raw scores to positive labels and negative labels. 
To process sentences with BERT-style '[CLS]', '[SEP]' tokens, you can use `data.BERTSentenceTransform` API.

In [9]:
train_dataset_raw = nlp.data.IMDB('train')
test_dataset_raw = nlp.data.IMDB('test')

tokenizer = nlp.data.BERTTokenizer(vocabulary)

def transform_fn(data):
    text, label = data
    # Transform label into position / negative
    label = 1 if label >= 5 else 0
    transform = nlp.data.BERTSentenceTransform(tokenizer, max_seq_length=128,
                                               pad=False, pair=False)
    data, length, segment_type = transform([text])
    data = data.astype('float32')
    length = length.astype('float32')
    segment_type = segment_type.astype('float32')
    return data, length, segment_type, label

In [10]:
train_dataset = train_dataset_raw.transform(transform_fn)
test_dataset = test_dataset_raw.transform(transform_fn)

print(vocabulary)
print('Index for [CLS] = ', vocabulary['[CLS]'])
print('Index for [SEP] = ', vocabulary['[SEP]'])

data, length, segment_type, label = train_dataset[0]
print('words = ', data.astype('int32'))

Vocab(size=30522, unk="[UNK]", reserved="['[CLS]', '[SEP]', '[MASK]', '[PAD]']")
Index for [CLS] =  2
Index for [SEP] =  3
words =  [    2 22953  2213  4381  2152  2003  1037  9476  4038  1012  2009  2743
  2012  1996  2168  2051  2004  2070  2060  3454  2055  2082  2166  1010
  2107  2004  1000  5089  1000  1012  2026  3486  2086  1999  1996  4252
  9518  2599  2033  2000  2903  2008 22953  2213  4381  2152  1005  1055
 18312  2003  2172  3553  2000  4507  2084  2003  1000  5089  1000  1012
  1996 25740  2000  5788 13732  1010  1996 12369  3993  2493  2040  2064
  2156  2157  2083  2037 17203  5089  1005 13433  8737  1010  1996  9004
 10196  4757  1997  1996  2878  3663  1010  2035 10825  2033  1997  1996
  2816  1045  2354  1998  2037  2493  1012  2043  1045  2387  1996  2792
  1999  2029  1037  3076  8385  2699  2000  6402  2091  1996  2082  1010
  1045  3202  7383  1012  1012  1012  1012     3]


### Batchify and Data Loader

In [11]:
padding_id = vocabulary[vocabulary.padding_token]
batchify_fn = nlp.data.batchify.Tuple(
        # words: the first dimension is the batch dimension
        nlp.data.batchify.Pad(axis=0, pad_val=padding_id),
        # valid length
        nlp.data.batchify.Stack(),
        # segment type : the first dimension is the batch dimension
        nlp.data.batchify.Pad(axis=0, pad_val=padding_id),
        # label
        nlp.data.batchify.Stack(np.float32))

batch_size = 32 * len(ctx)
train_data = gluon.data.DataLoader(train_dataset,
                                   batchify_fn=batchify_fn, shuffle=True,
                                   batch_size=batch_size, num_workers=4)
test_data = gluon.data.DataLoader(test_dataset,
                                  batchify_fn=batchify_fn,
                                  shuffle=False, batch_size=batch_size, num_workers=4)

### Training Loop

Now we have all the pieces to put together, and we can finally start fine-tuning the
model with a few epochs.

In [12]:
num_epoch = 1
lr = 0.00005
train_loop(net, train_data, test_data, num_epoch, lr, ctx, loss_fn)

Batch 0 Acc 0.375 Train Loss 0.820992186665535
Batch 25 Acc 0.6859975961538461 Train Loss 0.5570519593759224
Batch 50 Acc 0.742953431372549 Train Loss 0.4945955764429242
Batch 75 Acc 0.7708675986842105 Train Loss 0.45561060265294817
Batch 100 Acc 0.7896813118811881 Train Loss 0.42588772823905
Batch 125 Acc 0.8024553571428571 Train Loss 0.4109316961691966
Batch 150 Acc 0.8158629966887417 Train Loss 0.3893569020788796
Batch 175 Acc 0.82470703125 Train Loss 0.37523524420843885
Epoch 0, Acc ('accuracy', 0.82932), Train Loss 0.3663616171391795
Test Acc 0.8854353768484933


### Prediction

In [13]:
predict_sentiment(net, ctx, vocabulary, tokenizer, 'this movie is so great')

'positive'

## Conclusion

In this tutorial, we showed how to fine-tune sentiment analysis model with pre-trained BERT parameters. In GluonNLP, this can be done with such few, simple steps. All we did was apply a BERT-style data transformation to pre-process the data, automatically download the pre-trained model, and feed the transformed data into the model, all within 50 lines of code!

For more fine-tuning scripts, visit the [BERT model zoo webpage](http://gluon-nlp.mxnet.io/model_zoo/bert/index.html).

## References

[1] Devlin, Jacob, et al. "Bert:
Pre-training of deep
bidirectional transformers for language understanding."
arXiv preprint
arXiv:1810.04805 (2018).

[2] Dolan, William B., and Chris
Brockett.
"Automatically constructing a corpus of sentential paraphrases."
Proceedings of
the Third International Workshop on Paraphrasing (IWP2005). 2005.

[3] Peters,
Matthew E., et al. "Deep contextualized word representations." arXiv
preprint
arXiv:1802.05365 (2018).

[4] Hendrycks, Dan, and Kevin Gimpel. "Gaussian error linear units (gelus)." arXiv preprint arXiv:1606.08415 (2016).

For fine-tuning, we only need to initialize the last classifier layer from scratch. The other layers are already initialized from the pre-trained model weights.